# 🔥 Générateur de Heatmaps - Test Google Colab

**Ce notebook vous permet de tester le générateur SANS RIEN installer sur votre ordinateur !**

## 📋 Instructions

1. Exécutez les cellules **dans l'ordre** (bouton Play ▶️ ou Shift+Enter)
2. Uploadez votre fichier Excel quand demandé
3. Téléchargez le PowerPoint généré !

---

## 📦 Étape 1 : Installation des bibliothèques

**Temps : ~30 secondes**

In [ ]:
# Installation des bibliothèques nécessaires
!pip install openpyxl python-pptx scipy -q

print("✅ Bibliothèques installées !")

## 🔧 Étape 2 : Code du générateur

**Exécutez cette cellule pour charger le moteur**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from pptx import Presentation
from pptx.util import Inches
from io import BytesIO
import warnings
warnings.filterwarnings('ignore')

class HeatmapGenerator:
    """
    Générateur de heatmaps ultra-flexible
    """
    
    def __init__(self, fichier_csv, config=None):
        self.fichier_csv = fichier_csv
        self.config = config or {}
        self.data = None
        self.marqueurs = []
        self.valeurs_x = []
        self.groupes = []
        self.nom_colonne_x = None
        
        # Configuration par défaut
        self.default_config = {
            'colonne_x': None,
            'label_axe_x': None,
            'echelle_log': True,
            'taille_heatmap': (3.5, 4),
            'dpi': 150,
            'max_heatmaps_par_slide': 6,
            'titre_presentation': 'Analyse des résultats expérimentaux',
            'sous_titre': 'Heatmaps - Données quantitatives'
        }
        
        for key, value in self.default_config.items():
            if key not in self.config:
                self.config[key] = value
    
    def charger_donnees(self):
        print("\n" + "="*80)
        print("ANALYSE DU FICHIER")
        print("="*80 + "\n")
        
        if self.fichier_csv.endswith('.xlsx') or self.fichier_csv.endswith('.xls'):
            try:
                df = pd.read_excel(self.fichier_csv, sheet_name='Données', engine='openpyxl')
            except:
                df = pd.read_excel(self.fichier_csv, sheet_name=0, engine='openpyxl')
        else:
            df = pd.read_csv(self.fichier_csv)
        
        print(f"✓ Fichier chargé : {self.fichier_csv}")
        
        if self.config['colonne_x'] is None:
            colonne_x = df.columns[0]
        else:
            colonne_x = self.config['colonne_x']
        
        self.nom_colonne_x = colonne_x
        
        if colonne_x in df.columns:
            df = df.dropna(subset=[colonne_x])
            df = df[df[colonne_x].astype(str).str.len() < 20]
            df = df[~df[colonne_x].astype(str).str.contains('📝|INSTRUCTION|Note|INFO', case=False, na=False)]
        
        self.valeurs_x = df[colonne_x].tolist()
        colonnes_donnees = [col for col in df.columns if col != colonne_x]
        
        self.marqueurs = self._detecter_marqueurs(colonnes_donnees)
        self.groupes = self._detecter_groupes(colonnes_donnees, self.marqueurs)
        
        print(f"  - {len(self.valeurs_x)} valeurs : {self.valeurs_x}")
        print(f"  - {len(self.marqueurs)} marqueurs : {self.marqueurs}")
        print(f"  - {len(self.groupes)} groupes : {self.groupes}")
        
        self.data = df
    
    def _detecter_marqueurs(self, colonnes):
        marqueurs = set()
        for col in colonnes:
            for sep in ['_', '-', '.']:
                if sep in col:
                    marqueur = col.split(sep)[0]
                    marqueurs.add(marqueur)
                    break
        return sorted(list(marqueurs))
    
    def _detecter_groupes(self, colonnes, marqueurs):
        groupes = set()
        for col in colonnes:
            for sep in ['_', '-', '.']:
                if sep in col:
                    parties = col.split(sep)
                    if len(parties) >= 2:
                        marqueur = parties[0]
                        if marqueur in marqueurs:
                            groupe = parties[1]
                            if not groupe.isdigit():
                                groupes.add(groupe)
                    break
        return sorted(list(groupes))
    
    def calculer_matrices(self):
        print("\n" + "="*80)
        print("CALCUL DES MATRICES")
        print("="*80 + "\n")
        
        self.matrices = {}
        colonne_x = self.nom_colonne_x
        
        for marqueur in self.marqueurs:
            matrice = np.zeros((len(self.valeurs_x), len(self.groupes)))
            
            for j, groupe in enumerate(self.groupes):
                colonnes_replicats = []
                for col in self.data.columns:
                    if col != colonne_x:
                        for sep in ['_', '-', '.']:
                            if sep in col:
                                parties = col.split(sep)
                                if len(parties) >= 2:
                                    if parties[0] == marqueur and parties[1] == groupe:
                                        colonnes_replicats.append(col)
                                break
                
                for i, val_x in enumerate(self.valeurs_x):
                    valeurs_replicats = []
                    for col_rep in colonnes_replicats:
                        val = self.data.loc[self.data[colonne_x] == val_x, col_rep].values
                        if len(val) > 0 and not np.isnan(val[0]):
                            valeurs_replicats.append(val[0])
                    
                    if valeurs_replicats:
                        matrice[i, j] = np.mean(valeurs_replicats)
                    else:
                        matrice[i, j] = 0
            
            self.matrices[marqueur] = matrice
            print(f"✓ {marqueur:<20} : matrice {len(self.valeurs_x)}×{len(self.groupes)}")
    
    def creer_heatmap(self, marqueur, afficher_valeurs=True, palette='rouge'):
        matrice = self.matrices[marqueur]
        
        if self.config['echelle_log']:
            matrice_plot = np.log10(matrice + 1)
        else:
            matrice_plot = matrice
        
        palettes = {
            'rouge': ['#FFFFFF', '#FFF5E6', '#FFE6CC', '#FFD9B3', '#FFCC99', 
                     '#FFB366', '#FF9933', '#FF8000', '#E67300', '#CC6600',
                     '#B35900', '#994C00', '#803F00'],
            'bleu': ['#FFFFFF', '#E6F2FF', '#CCE5FF', '#B3D9FF', '#99CCFF',
                    '#80BFFF', '#66B3FF', '#4DA6FF', '#3399FF', '#1A8CFF',
                    '#0080FF', '#0073E6', '#0066CC'],
            'viridis': plt.cm.viridis(np.linspace(0, 1, 13)),
            'plasma': plt.cm.plasma(np.linspace(0, 1, 13))
        }
        
        if palette in ['viridis', 'plasma']:
            cmap = LinearSegmentedColormap.from_list(palette, palettes[palette], N=100)
        else:
            cmap = LinearSegmentedColormap.from_list(palette, palettes[palette], N=100)
        
        fig, ax = plt.subplots(figsize=self.config['taille_heatmap'], dpi=self.config['dpi'])
        im = ax.imshow(matrice_plot, cmap=cmap, aspect='auto')
        
        ax.set_xticks(np.arange(len(self.groupes)))
        ax.set_yticks(np.arange(len(self.valeurs_x)))
        ax.set_xticklabels(self.groupes, fontsize=9)
        ax.set_yticklabels(self.valeurs_x, fontsize=9)
        
        label_x = self.config['label_axe_x'] if self.config['label_axe_x'] else self.nom_colonne_x
        ax.set_xlabel(label_x, fontsize=10, fontweight='bold')
        
        plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")
        
        if afficher_valeurs:
            for i in range(len(self.valeurs_x)):
                for j in range(len(self.groupes)):
                    val = matrice[i, j]
                    if val < 1:
                        text = f'{val:.2f}'
                    elif val < 10:
                        text = f'{val:.1f}'
                    else:
                        text = f'{val:.0f}'
                    
                    if matrice_plot[i, j] > (matrice_plot.max() * 0.6):
                        color = 'white'
                    else:
                        color = 'black'
                    
                    ax.text(j, i, text, ha="center", va="center", color=color, fontsize=8)
        
        ax.set_title(marqueur, fontsize=12, fontweight='bold', pad=10)
        
        cbar = plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
        if self.config['echelle_log']:
            cbar.set_label('log₁₀(valeur + 1)', rotation=270, labelpad=15, fontsize=8)
        else:
            cbar.set_label('Valeur', rotation=270, labelpad=15, fontsize=8)
        
        plt.tight_layout()
        
        buffer = BytesIO()
        plt.savefig(buffer, format='png', bbox_inches='tight', dpi=self.config['dpi'])
        buffer.seek(0)
        plt.close()
        
        return buffer
    
    def creer_presentation(self, fichier_sortie, afficher_valeurs=True, palette='rouge'):
        print("\n" + "="*80)
        print("CRÉATION POWERPOINT")
        print("="*80 + "\n")
        
        prs = Presentation()
        prs.slide_width = Inches(10)
        prs.slide_height = Inches(7.5)
        
        # Slide titre
        slide = prs.slides.add_slide(prs.slide_layouts[6])
        txBox = slide.shapes.add_textbox(Inches(1), Inches(2.5), Inches(8), Inches(1))
        tf = txBox.text_frame
        tf.text = self.config['titre_presentation']
        tf.paragraphs[0].font.size = Inches(0.4)
        tf.paragraphs[0].font.bold = True
        
        print("✓ Slide 1 : Titre")
        
        # Slides heatmaps
        max_par_slide = self.config['max_heatmaps_par_slide']
        if max_par_slide <= 4:
            cols, rows = 2, 2
        elif max_par_slide <= 6:
            cols, rows = 3, 2
        else:
            cols, rows = 3, 3
        
        heatmap_width = 9 / cols
        heatmap_height = 6 / rows
        
        num_slides = 0
        for i in range(0, len(self.marqueurs), max_par_slide):
            batch = self.marqueurs[i:i + max_par_slide]
            slide = prs.slides.add_slide(prs.slide_layouts[6])
            
            for j, marqueur in enumerate(batch):
                row = j // cols
                col = j % cols
                left = Inches(0.5 + col * heatmap_width)
                top = Inches(1 + row * heatmap_height)
                
                img_buffer = self.creer_heatmap(marqueur, afficher_valeurs, palette)
                slide.shapes.add_picture(img_buffer, left, top,
                                        width=Inches(heatmap_width * 0.9),
                                        height=Inches(heatmap_height * 0.85))
            num_slides += 1
        
        print(f"✓ Slides 2-{num_slides+1} : {len(self.marqueurs)} heatmaps")
        
        prs.save(fichier_sortie)
        print(f"\n✅ PowerPoint créé : {fichier_sortie}")

print("✅ Générateur de heatmaps chargé !")

## 📤 Étape 3 : Upload de votre fichier Excel

**Cliquez sur le bouton pour choisir votre fichier**

In [ ]:
from google.colab import files

print("📤 Cliquez sur 'Choisir les fichiers' et sélectionnez votre Excel...")
uploaded = files.upload()

# Récupérer le nom du fichier
nom_fichier = list(uploaded.keys())[0]
print(f"\n✅ Fichier uploadé : {nom_fichier}")

## ⚙️ Étape 4 : Configuration

**Personnalisez vos options ici**

In [ ]:
# ===== CONFIGURATION - MODIFIEZ ICI =====

# Palette de couleurs
PALETTE = 'rouge'  # Choix: 'rouge', 'bleu', 'viridis', 'plasma'

# Afficher les valeurs dans les cellules
AFFICHER_VALEURS = True  # True ou False

# Configuration avancée
CONFIG = {
    'colonne_x': None,  # None = détection auto
    'label_axe_x': None,  # Personnalisez l'étiquette de l'axe
    'echelle_log': True,  # True = log, False = linéaire
    'titre_presentation': 'Test Générateur Heatmaps',
    'max_heatmaps_par_slide': 6
}

print("✅ Configuration prête !")
print(f"  - Palette : {PALETTE}")
print(f"  - Valeurs affichées : {AFFICHER_VALEURS}")
print(f"  - Échelle log : {CONFIG['echelle_log']}")

## 🚀 Étape 5 : GÉNÉRATION !

**Exécutez cette cellule pour créer votre PowerPoint**

In [ ]:
# Création du générateur
generator = HeatmapGenerator(nom_fichier, CONFIG)

# Chargement des données
generator.charger_donnees()

# Calcul des matrices
generator.calculer_matrices()

# Création du PowerPoint
fichier_sortie = 'Heatmaps_Test_Colab.pptx'
generator.creer_presentation(fichier_sortie, 
                            afficher_valeurs=AFFICHER_VALEURS,
                            palette=PALETTE)

print("\n" + "="*80)
print("✅ TERMINÉ !")
print("="*80)

## 📥 Étape 6 : Télécharger le résultat

**Téléchargez votre PowerPoint !**

In [ ]:
from google.colab import files

print("📥 Téléchargement du PowerPoint...")
files.download(fichier_sortie)
print("\n✅ Téléchargement lancé ! Vérifiez vos téléchargements.")

---

## 🎉 C'EST FINI !

### Vous avez testé le générateur sans rien installer !

**Si ça vous plaît, vous pouvez maintenant :**
1. L'installer sur votre PC (avec Anaconda recommandé)
2. Continuer à l'utiliser sur Colab (gratuit !)
3. Tester avec d'autres fichiers Excel

**Pour changer les couleurs :**
- Modifiez `PALETTE` dans l'Étape 4
- Choix : `'rouge'`, `'bleu'`, `'viridis'`, `'plasma'`

**Pour réutiliser ce notebook :**
- Sauvegardez-le dans votre Google Drive
- Vous pourrez le réutiliser à volonté !

---

**Questions ?** Contactez-moi ! 😊